<a href="https://colab.research.google.com/github/Bayzid03/gemini-chain-of-minds/blob/main/gemini_chain_of_minds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-generativeai

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

def configure():
    secret_name = "GOOGLE_API_KEY"

    api_key = userdata.get(secret_name)
    if not api_key:
        raise ValueError(f"Secret '{secret_name}' not found. Please check the name in Colab's Secrets.")

    genai.configure(api_key=api_key)
    return genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
def planner_agent(model, topic: str) -> list[str]:
    print("Planner Agent: Creating a research plan...")
    prompt = f"""
    You are an expert research planner. Your task is to break down the following topic
    into 3-5 specific, answerable questions. Return these questions as a Python list of strings.

    TOPIC: "{topic}"

    Example output: ["question 1", "question 2", "question 3"]
    """

    try:
        response = model.generate_content(prompt)
        plan_str = response.text.strip().replace('[', '').replace(']', '').replace('"', '')
        plan = [q.strip() for q in plan_str.split(',') if q.strip()]

        print("Plan created:")
        for i, q in enumerate(plan, 1):
            print(f"   {i}. {q}")
        return plan
    except Exception as e:
        print(f"Error in Planner Agent: {e}")
        return []